In [18]:
import os
import json

import pandas as pd
from tqdm.notebook import tqdm

from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
x = torch.rand(5, 3)
print(x)
torch.cuda.is_available()

tensor([[0.1193, 0.7770, 0.4199],
        [0.6059, 0.2519, 0.5031],
        [0.5755, 0.3480, 0.7119],
        [0.2364, 0.3714, 0.1486],
        [0.1595, 0.5929, 0.5557]])


False

In [10]:
MODEL_NAME = "Qwen/Qwen2.5-1.5B-Instruct"

In [11]:
def load_model(model_name):
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        dtype="auto",
        device_map="auto"
    )
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    return model, tokenizer

In [12]:
def chat(prompt, model, tokenizer):

    messages = [
        {"role": "system", "content": "You are a social media content generator. You are gonna create only the post content based on the user's prompt."},
        {"role": "user", "content": "Like a user of social media platform. You are not gonna use emojis or hashtags."},
        {"role": "user", "content": prompt}
    ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=512
    )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]

    return tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

In [13]:
model, tokenizer = load_model(MODEL_NAME)

def generate_post(lang, keyword, model=model, tokenizer=tokenizer):
    
    chat_output = chat(f"Write a post in {lang} about {keyword}", model, tokenizer)
    return chat_output.replace('"', '')

'(MaxRetryError('HTTPSConnectionPool(host=\'huggingface.co\', port=443): Max retries exceeded with url: /Qwen/Qwen2.5-1.5B-Instruct/resolve/main/config.json (Caused by NameResolutionError("HTTPSConnection(host=\'huggingface.co\', port=443): Failed to resolve \'huggingface.co\' ([Errno 11001] getaddrinfo failed)"))'), '(Request ID: 45a8db3e-6aa5-44cd-8a55-cea99517103e)')' thrown while requesting HEAD https://huggingface.co/Qwen/Qwen2.5-1.5B-Instruct/resolve/main/config.json
Retrying in 1s [Retry 1/5].
'(MaxRetryError('HTTPSConnectionPool(host=\'huggingface.co\', port=443): Max retries exceeded with url: /Qwen/Qwen2.5-1.5B-Instruct/resolve/main/config.json (Caused by NameResolutionError("HTTPSConnection(host=\'huggingface.co\', port=443): Failed to resolve \'huggingface.co\' ([Errno 11001] getaddrinfo failed)"))'), '(Request ID: c86b9ec5-a6ca-4cfc-ab12-61e1f154b22a)')' thrown while requesting HEAD https://huggingface.co/Qwen/Qwen2.5-1.5B-Instruct/resolve/main/config.json
Retrying in 2s [

In [14]:
KEYWORDS_FILE = "../keyword_groups.json"
with open(KEYWORDS_FILE, "r", encoding="utf-8") as f:
    KEYWORD_GROUPS = json.load(f)
print(KEYWORD_GROUPS)

{'English': {'lang': 'en', 'keywords': ['green transition', 'greenhouse effect', 'loss of biodiversity', 'extreme weather events', 'co2', 'emissions', 'global warming', 'melting glaciers', 'renewable energy', 'misinformation', 'catastrophe', 'anthropogenic ', 'sustainable biofuels', 'desertification', 'deforestation', 'clean energy', 'sea level rise', 'extinction']}, 'Spain': {'lang': 'es', 'keywords': ['transición verde', 'efecto invernadero', 'pérdida de biodiversidad', 'eventos climáticos extremos', 'co2', 'emisiones', 'calentamiento global', 'derretimiento de glaciares', 'energía renovable', 'desinformación', 'catástrofe', 'antropogénico', 'biocombustibles sostenibles', 'desertificación', 'deforestación', 'energía limpia', 'aumento del nivel del mar', 'extinción']}, 'Portugal': {'lang': 'pt', 'keywords': ['transição verde', 'efeito estufa', 'perda de biodiversidade', 'eventos climáticos extremos', 'co2', 'emissões', 'aquecimento global', 'derretimento das geleiras', 'energia renová

In [15]:
OUTPUT_DIR = "./synthetic_data/"
os.makedirs(OUTPUT_DIR, exist_ok=True)

In [17]:
def generate_synthetic_csvs(groups, posts_per_keyword=1000):
    """
    Generates a CSV per language containing synthetic social media posts.
    """
    for region, info in groups.items():
        lang = info['lang']
        keywords = info['keywords']
        posts_data = []
        
        print(f"Generating posts for {region} ({lang})...")
        
        for keyword in tqdm(keywords, desc=f"Keywords ({region})"):
            # Loop to simulate up to 1000 posts per keyword
            for _ in tqdm(range(posts_per_keyword), desc=f"Generating posts for keyword: {keyword}", leave=False):                
                content = generate_post(lang=lang, keyword=keyword)
                posts_data.append({
                    "content": content,
                    "keyword": keyword
                })
        
        # Save DataFrame to CSV
        df = pd.DataFrame(posts_data)
        filename = f"{OUTPUT_DIR}synthetic_posts_{lang}.csv"
        df.to_csv(filename, index=False)
        print(f"Saved {filename} with {len(df)} posts.")

generate_synthetic_csvs(KEYWORD_GROUPS)


Generating posts for English (en)...


Keywords (English):   0%|          | 0/18 [00:00<?, ?it/s]

Generating posts for keyword: green transition:   0%|          | 0/1000 [00:00<?, ?it/s]

KeyboardInterrupt: 